In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# !wget https://github.com/explosion/spacy-models/releases/download/en_vectors_web_lg-2.1.0/en_vectors_web_lg-2.1.0.tar.gz -O en_vectors_web_lg-2.1.0.tar.gz
# !pip install -q en_vectors_web_lg-2.1.0.tar.gz

https://github.com/MILVLG/openvqa/blob/6b9bfeb2e6462b946d7e7866ffc49dd7a8bcece3/openvqa/core/base_cfgs.py
https://github.com/MILVLG/openvqa/blob/6b9bfeb2e6462b946d7e7866ffc49dd7a8bcece3/utils/exec.py#L36
https://github.com/MILVLG/openvqa/blob/6b9bfeb2e6462b946d7e7866ffc49dd7a8bcece3/utils/test_engine.py#L49
https://github.com/MILVLG/openvqa/blob/6b9bfeb2e6462b946d7e7866ffc49dd7a8bcece3/openvqa/datasets/vqa/vqa_loader.py#L68

In [25]:
import gc
gc.collect()

6133

In [32]:
import json

import numpy as np
import torch
import torch.utils.data as Data
import yaml
from yaml import CLoader

from openvqa.models.model_loader import CfgLoader
from run import create_parser
from utils.test_engine import test_engine

parser = create_parser()
arg = '--RUN val --MODEL ban_8 --DATASET vqa --NW 2 --CKPT_V ban_8 --CKPT_E 13'
args = parser.parse_args(arg.split(' '))

cfg_file = "configs/{}/{}.yml".format(args.DATASET, args.MODEL)
with open(cfg_file, 'r') as f:
    yaml_dict = yaml.load(f, Loader=CLoader)

__C = CfgLoader(yaml_dict['MODEL_USE']).load()
args = __C.str_to_bool(args)
args_dict = __C.parse_to_dict(args)

args_dict = {**yaml_dict, **args_dict}
__C.add_args(args_dict)
__C.proc()

print('Hyper Parameters:')
print(__C)

Checking dataset ........
Finished!

Hyper Parameters:
{ BATCH_SIZE        }->512
{ BA_HIDDEN_SIZE    }->3072
{ BBOX_NORMALIZE    }->False
{ CACHE_PATH        }->./results/cache
{ CKPTS_PATH        }->./ckpts
{ CKPT_EPOCH        }->13
{ CKPT_PATH         }->None
{ CKPT_VERSION      }->ban_8
{ CLASSIFER_DROPOUT_R }->0.5
{ DATASET           }->vqa
{ DATA_PATH         }->{'vqa': './data/vqa', 'gqa': './data/gqa', 'clevr': './data/clevr'}
{ DATA_ROOT         }->./data
{ DEVICES           }->[0]
{ DROPOUT_R         }->0.2
{ EVAL_BATCH_SIZE   }->32
{ EVAL_EVERY_EPOCH  }->False
{ FEATS_PATH        }->{'vqa': {'train': './data/vqa/feats/train2014', 'val': './data/vqa/feats/val2014', 'test': './data/vqa/feats/test2015'}, 'gqa': {'default-frcn': './data/gqa/feats/gqa-frcn', 'default-grid': './data/gqa/feats/gqa-grid'}, 'clevr': {'train': './data/clevr/feats/train', 'val': './data/clevr/feats/val', 'test': './data/clevr/feats/test'}}
{ FEAT_SIZE         }->{'vqa': {'FRCN_FEAT_SIZE': (100, 2048), 

In [26]:
from openvqa.datasets.dataset_loader import DatasetLoader, EvalLoader

dataset = DatasetLoader(__C).DataSet()

data_size = dataset.data_size
token_size = dataset.token_size
ans_size = dataset.ans_size
pretrained_emb = dataset.pretrained_emb

 ========== Dataset size: 214354
 ========== Question token vocab size: 20573
 ========== Answer token vocab size (occur more than 8 times): 3129
Finished!



In [4]:
from openvqa.models.model_loader import ModelLoader

net = ModelLoader(__C).Net(
    __C,
    pretrained_emb,
    token_size,
    ans_size
)

# net.cuda()
net.eval()

with open('ckpts/ckpt_ban_8/epoch13.pkl', 'rb') as f:
    state = torch.load(f, map_location=torch.device('cpu'))
net.load_state_dict(state['state_dict'])

In [6]:
dataloader = Data.DataLoader(
    dataset,
    batch_size=__C.EVAL_BATCH_SIZE,
    shuffle=False,
    num_workers=__C.NUM_WORKERS,
    pin_memory=__C.PIN_MEM
)

In [7]:
%%time
ans_ix_list = []
pred_list = []

for step, (
        frcn_feat_iter,
        grid_feat_iter,
        bbox_feat_iter,
        ques_ix_iter,
        ans_iter
    ) in enumerate(dataloader):
    print("\rEvaluation: [step %4d/%4d]" % (
        step,
        int(data_size / __C.EVAL_BATCH_SIZE),
    ), end='          ')
    pred = net(
        frcn_feat_iter,
        grid_feat_iter,
        bbox_feat_iter,
        ques_ix_iter
    )
    pred_np = pred.data.numpy()
    pred_argmax = np.argmax(pred_np, axis=1)
    
    if pred_argmax.shape[0] != __C.EVAL_BATCH_SIZE:
        pred_argmax = np.pad(
            pred_argmax,
            (0, __C.EVAL_BATCH_SIZE - pred_argmax.shape[0]),
            mode='constant',
            constant_values=-1
        )

    ans_ix_list.append(pred_argmax)
    
    if __C.TEST_SAVE_PRED:
        if pred_np.shape[0] != __C.EVAL_BATCH_SIZE:
            pred_np = np.pad(
                pred_np,
                ((0, __C.EVAL_BATCH_SIZE - pred_np.shape[0]), (0, 0)),
                mode='constant',
                constant_values=-1
            )

        pred_list.append(pred_np)
    if step > 10:
        break

Evaluation: [step   11/13993]          CPU times: user 1min 12s, sys: 4.35 s, total: 1min 16s
Wall time: 39.4 s


In [8]:
ans_ix_list = np.array(ans_ix_list).reshape(-1)
ans_ix_list

array([   9,   66,    9,    3,    3, 2472,  232,    4,  785,  233,    9,
          9,    3,    3,    9,    3, 1273,  288,    9, 1535,  318,   30,
        758,  725,    9,    9,    9,   32,   12,    9,    9, 2518,   59,
          3,    9,    4, 1075,    9,   17,  593,    9,  448,  418,  286,
         13, 1303,    3,    9,    3, 1011,  438,   10,    3,    4,    3,
         40,    4,   17,    9,    9,    9,    3,    9,    4,  168,   59,
          3,  257,    3,  645,  645,  257,  237,  257,  208,  371,  371,
        395,    3,    9,   65,    3,  672,   82,  232,  296,    9,  821,
         83,   74, 1096,    9,    3,    9,  514,   16,  268,   30,    9,
        891,   60,    9,    9, 1009, 2306,    9,    9,    9,  392, 1680,
          3, 1468,   75,    3, 1016,   30,    9,    9,    3,   17,    3,
        829,  295, 1186,  672,  120,  221,    9,    3,    3,    6,    3,
          9,    9,    9,  232,  268,    9,   30,    3,    9,    3,    9,
        252, 1938,  176,    3,  655,   53,    9,   

In [34]:
qid_list = [ques['question_id'] for ques in dataset.ques_list]
ans_size = dataset.ans_size

In [35]:
result = [{
        'answer': dataset.ix_to_ans[str(ans_ix_list[qix])],
        'question_id': int(qid_list[qix])
    } for qix in range(ans_ix_list.__len__())]

In [36]:
result_eval_file = __C.CACHE_PATH + '/result_run_' + __C.VERSION
log_file = __C.LOG_PATH + '/log_run_' + __C.VERSION + '.txt'

print('Save the result to file: {}'.format(result_eval_file))
with open(result_eval_file, 'w') as f:
    json.dump(result, f)

Save the result to file: ./results/cache/result_run_2666747


In [37]:
from openvqa.datasets.vqa.eval.vqa import VQA

ques_file_path = __C.RAW_PATH[__C.DATASET][__C.SPLIT['val']]
ans_file_path = __C.RAW_PATH[__C.DATASET][__C.SPLIT['val'] + '-anno']
# uncomment line 165-166 in http://localhost:8888/edit/openvqa/openvqa/datasets/vqa/eval/vqa.py
vqa = VQA(ans_file_path, ques_file_path)
vqaRes = vqa.loadRes(result_eval_file, ques_file_path)

loading VQA annotations and questions into memory...
0:00:07.463692
creating index...
index created!
Loading and preparing results...     
DONE (t=0.00s)
creating index...
index created!


In [39]:
from openvqa.datasets.vqa.eval.vqaEval import VQAEval
vqaEval = VQAEval(vqa, vqaRes, n=2)

In [43]:
quesIds = [d['question_id'] for d in result]

In [45]:
vqaEval.evaluate(quesIds)

computing accuracy
Finished Percent: [################----] 78% Done computing accuracy


In [46]:
vqaEval.accuracy

{'overall': 10.36,
 'perQuestionType': {'none of the above': 28.57,
  'what are the': 0.0,
  'what is': 0.0,
  'what': 0.0,
  'is this a': 7.69,
  'is this': 15.71,
  'what is the man': 0.0,
  'how many': 0.0,
  'what does the': 0.0,
  'why': 3.75,
  'is it': 16.25,
  'why is the': 0.0,
  'what color is the': 0.0,
  'is there a': 11.11,
  'is the': 27.19,
  'is that a': 50.0,
  'are these': 0.0,
  'are the': 36.0,
  'what is the': 0.0,
  'which': 0.0,
  'could': 0.0,
  'are there': 20.0,
  'what kind of': 0.0,
  'has': 0.0,
  'what color are the': 0.0,
  'are there any': 50.0,
  'is this person': 63.33,
  'does the': 12.5,
  'where is the': 0.0,
  'how many people are': 0.0,
  'can you': 0.0,
  'what type of': 0.0,
  'what is the color of the': 0.0,
  'what is on the': 0.0,
  'does this': 72.5,
  'is this an': 66.67,
  'who is': 0.0,
  'what is this': 0.0,
  'is there': 43.33,
  'are': 33.33,
  'what time': 0.0,
  'how': 0.0,
  'what is in the': 0.0,
  'do you': 0.0,
  'what are': 0.0,